# Tutorial 6.4 - Orientation Selectivity in a Ring Model
### Neuroscience goal:
- see how contrast-invariant gain can arise from different network configurations
### Computational goal:
- simulate a large number of coupled ODEs, using arrays to store variables and their coupling constants

### Overview
- will simulate a network of 100 firing-rate model units (50 excitatory, 50 inhibitory)
    - to investigate how connections within the network shape neural responses to external input
- will compare feedforward model with recurrent circuit model

set up two arrays, with `Nt` rows and 50 columns, where `Nt` is number of time points to be simulated
- one array for excitatory cells and the other for inhibitory cells
- simulation of 300ms with 0.1ms is sufficient

In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
dt = 0.1e-3
t = np.arange(0, 0.3, dt)

In [7]:
exc = np.zeros((len(t), 50))
inh = np.zeros((len(t), 50))

define three 50x50 connectivity matrices, each of which can be initialized with entries of zero
- these are $W^{EE}, W^{EI}, W^{IE}$ (we ignore I-I connections in this tutorial)

In [8]:
w_ee = np.zeros((50, 50))
w_ei = np.zeros((50, 50))
w_ie = np.zeros((50, 50))

their firing rates will be treated as a linear function of inputs, dynamics following:

$\tau _E \frac{dr_i^E}{dt} = -r_i^E + \frac{1}{N} \sum _j W_{ij}^{EE}r_j^E + \frac{1}{N}\sum _j W_{ij}^{IE}r_j^I + I_0^E + S_i^E $

$ \tau_ I \frac{dr_i^I}{dt} = -r_i^I + \frac{1}{N}\sum _j W_{ij}^{EI}r_j^E + I_0^I + S_i^I $

with additional conditions:

$r_i^E > 0$ and $r_i^I > 0$ for all units and all times 
    
in the preceeding equtions, $I_0^E$ and $I_0^I$ represnet baseline input to excitatory and inihibitory units, respectively
- a negative value for these quantities is equivalent to a positive threshold on the f-I curve

the stimulus-dependent inputs, $S_i^E$ and $S_i^I$, depend on the orientation preference of the unit labeled $i$
- we define the orientation preference of each excitatory unit as: $\theta_i = \pi i / N$, where N=50 is the total number of units of a given type
- we define the orientatio of the stimulus using $\theta_{cue} = \pi / 2$ 

- given these definitions, we use: 

    $ S_i^E = A_Ec[1 + \epsilon * \cos(2\theta_{cue} - 2\theta_i)]  $

    $ S_i^I = A_Ic[1 + \epsilon * \cos(2\theta_{cue} - 2\theta_i)]  $

    - with $c$ being the contrast, which will vary from 0 to 1
    - $\epsilon = 0.5$ represents the degree of modulation of input with orientation
    - in this case producing a threefold increase in input from the null direction
    (where $\theta_{cue} = \theta_i \pi / 2$, so $\cos(2\theta_{cue} - 2\theta_i) = -1$) to the preferred direction (where $\theta_{cue} = \theta_i$, so $\cos(2\theta_{cue} - 2\theta_i) = +1$)

other parameters depend on the network being simulated as follows:
- network A:
    - $\tau_E = \tau_I = $ 10ms
    - $I_0^E = I_0^I = -10$
    - $A_E = 40$
    - $A_i^I = 40$
    - $W_{ij}^{EE} = W_{ij}^{EI} = W_{ij}^{IE} = 0$

- network B:
    - $\tau_E = \tau_I = $ 10ms
    - $I_0^E = I_0^I = -5$
    - $A_E = 40$
    - $A_i^I = 40$
    - $W_{ij}^{EE} = W_{ij}^{EI} = 0$
    - $W_{ij}^{IE} = -[1+\cos(\pi + 2\theta_i - 2\theta_j)] / N$

- network C:
    - $\tau_E = $ 50ms
    - $\tau_I = $ 5ms
    - $I_0^E = 2$
    - $I_0^I = 0.5$
    - $A_E = 100$
    - $A_i^I = 0$
    - $W_{ij}^{EE} = 5[1 +\cos(2\theta_i - 2\theta_j)] / N$
    - $W_{ij}^{EI} = 3[1 + \cos(2\theta_i - 2\theta_j)] / N$
    - $W_{ij}^{IE} = -4[1+\cos(\pi + 2\theta_i - 2\theta_j)] / N$

#### Complete questions 1-4 using all (A, B, C) networks
### 1. simulate network as function of time, with stimulus present
- with all firing rates initially set at zero, simulate the network as a function of time, with the stimulus present the entire time
- use contrasts of 0, 0.25, 0.50, 0.75, 1

- *note*: firing rates can be stored as arrays, with each row representing a different time point and each column a different cell
- allows to sum over all excitatory units providing input nto all excitatory cells within the network to be calculated using matrix multiplication
    - e.g., `rE[i-1, :] * W_EE`

- each entry in the row vector produced by the preceding segment of code represents the total within-network excitatory input to a unit
- the `i-1` represents the previous time point, because firing rates at the previous time point are used when calculating input at the current time point, i